In [1]:
import os
import random
from helper import *

In [2]:
images_folder = 'dataset/input/images'
labels_folder = 'dataset/input/labels'

In [3]:
class_mapping = {
    'apple': 0,
    'avocado': 1,
    'banana': 2,
    'kiwi': 3,
    'lemon': 4,
    'orange': 5,
    'pear': 6,
    'pomegranate': 7,
    'strawberry': 8,
    'watermelon': 9
}


#### Adding more images and labels to the dataset
[Fruit Images for Object Detection](https://www.kaggle.com/datasets/mbkinaci/fruit-images-for-object-detection)


In [ ]:
# Check and fix VOC XML sizes (width and height)
fix_voc_xml_sizes(
    "dataset/fruit_images (apple, banana, orange)",
    "dataset/fruit_images (apple, banana, orange)")

Fixed apple_1.xml: set width=349, height=349
No change for apple_10.xml
No change for apple_11.xml
No change for apple_12.xml
No change for apple_13.xml
No change for apple_14.xml
No change for apple_15.xml
No change for apple_16.xml
Fixed apple_17.xml: set width=700, height=800
No change for apple_18.xml
No change for apple_19.xml
No change for apple_2.xml
Fixed apple_20.xml: set width=500, height=500
No change for apple_21.xml
No change for apple_22.xml
No change for apple_23.xml
No change for apple_24.xml
No change for apple_25.xml
No change for apple_26.xml
No change for apple_27.xml
Fixed apple_28.xml: set width=300, height=300
No change for apple_29.xml
No change for apple_3.xml
No change for apple_30.xml
No change for apple_31.xml
No change for apple_32.xml
No change for apple_33.xml
No change for apple_35.xml
No change for apple_36.xml
Fixed apple_37.xml: set width=389, height=352
No change for apple_38.xml
No change for apple_39.xml
No change for apple_4.xml
No change for appl

In [ ]:
remove_verified_xml("dataset/fruit_images (apple, banana, orange)")

Deleted: dataset/fruit_images (apple, banana, orange)/apple_1.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_10.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_11.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_12.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_13.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_14.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_15.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_16.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_17.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_18.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_19.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_2.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_20.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_21.xml
Deleted: dataset/fruit_images (apple, banana, orange)/apple_22.x

In [ ]:
# Convert all VOC XML annotations to YOLO format, and remove XMLs with verified TXT
batch_convert_voc_to_yolo(
    "dataset/fruit_images (apple, banana, orange)",
    "dataset/fruit_images (apple, banana, orange)",
    class_mapping
)

In [ ]:
# Add more image samples and labels to the dataset
split_files_by_extension(
    'dataset/fruit_images (apple, banana, orange)',
    {
        '.jpg': images_folder,
        '.jpeg': images_folder,
        '.png': images_folder,
        '.txt': labels_folder,
        '.xml': labels_folder
    }
)

[Fruit Detection Dataset](https://www.kaggle.com/datasets/lakshaytyagi01/fruit-detection)

In [13]:
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

DATASET_PATH = "dataset/fruit-detection"
KAGGLE_DATASET = "lakshaytyagi01/fruit-detection"

if os.path.isdir(DATASET_PATH):
    print(f"Dataset already exists at {DATASET_PATH}, skipping download.")
else:
    print("Downloading dataset from Kaggle...")
    os.makedirs(DATASET_PATH, exist_ok=True)
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files(KAGGLE_DATASET, path=DATASET_PATH, unzip=False)
    # Find zip file
    for file in os.listdir(DATASET_PATH):
        if file.endswith(".zip"):
            zip_path = os.path.join(DATASET_PATH, file)
            with zipfile.ZipFile(zip_path, "r") as zip_ref:
                zip_ref.extractall(DATASET_PATH)
            os.remove(zip_path)
    print("Download and extraction complete.")


Dataset URL: https://www.kaggle.com/datasets/lakshaytyagi01/fruit-detection
Download and extraction complete.


Extract 150 pictures of watermelon from dataset

In [12]:
filter_and_extract_images_by_class(
    data_root_dir="dataset/fruit-detection/Fruits-detection",
    class_indices_to_keep={5},  # watermelon
    output_images_dir="dataset/unprocessed/images",
    output_labels_dir="dataset/unprocessed/labels",
    target_num_labels=150
)

Collected 150 labels/images with specified classes.


In [35]:
rename_images_and_labels(images_folder, labels_folder, prefix='img_')

In [36]:
delete_labels_without_images("dataset/unprocessed/labels", "dataset/unprocessed/images")

Deleted 0 label files without matching images:


In [13]:
relabel_yolo_labels("dataset/unprocessed/labels", {5: 9})  # Remap watermelon class (5) to (9)

Relabeled 150 label files.


In [ ]:
count_labels_per_class("dataset/input/labels", "dataset/input/images", class_mapping)

Total object counts by class:
Class 9 ("watermelon"): 353


In [15]:
split_files_by_extension(
    'dataset/unprocessed/images',
    {'.jpg': images_folder,
     '.jpeg': images_folder,
     '.png': images_folder})

split_files_by_extension(
    'dataset/unprocessed/labels',
    {'.txt': labels_folder})

#### Dataset labels by class

In [37]:
count_labels_per_class(labels_folder, images_folder, class_mapping)

Total object counts by class:
Class 0 ("apple"): 330
Class 1 ("avocado"): 307
Class 2 ("banana"): 11
Class 5 ("orange"): 155
Class 9 ("watermelon"): 592


In [38]:
count_files_by_extension(images_folder)
count_files_by_extension(labels_folder)

.jpg: 559
.jpeg: 128
.png: 112
.txt: 505


Counter({'.txt': 505})

#### Split Dataset into train/validation with equal class distribution

In [6]:
labels_folder = "dataset/input/labels"
images_folder = "dataset/input/images"
SEED = 42
random.seed(SEED)
n_classes = 10

In [39]:
# Count which classes are in each image
img_class_map = count_image_classes(labels_folder)
# Iterative stratified train/val+test split (75/15/10 split)
train_set, valtest_set = iterative_train_val_split(img_class_map.keys(), img_class_map, n_classes, val_ratio=0.25, seed=SEED)
valtest_class_map = {lbl: img_class_map[lbl] for lbl in valtest_set}
val_set, test_set = iterative_train_val_split(valtest_class_map.keys(), valtest_class_map, n_classes, val_ratio=0.4, seed=SEED)
# save splits
save_balanced_split(train_set, labels_folder, images_folder, "dataset/split/train/labels", "dataset/split/train/images")
save_balanced_split(val_set, labels_folder, images_folder, "dataset/split/val/labels", "dataset/split/val/images")
save_balanced_split(test_set, labels_folder, images_folder, "dataset/split/test/labels", "dataset/split/test/images")


In [ ]:
print("Training set class distribution before sampling:")
count_labels_per_class("dataset/split/train/images")
print("\n")
print("Validation set class distribution:")
count_labels_per_class("dataset/split/val/images")
print("\n")
print("Test set class distribution:")
count_labels_per_class("dataset/split/test/images")

Training set class distribution before sampling:
.jpg: 341
.jpeg: 33
.png: 4


Validation set class distribution:
.jpg: 70
.png: 1
.jpeg: 6


Test set class distribution:
.jpg: 45
.jpeg: 5


Counter({'.jpg': 45, '.jpeg': 5})

In [41]:
print("Training set class distribution before sampling:")
count_labels_per_class("dataset/split/train/labels", "dataset/split/train/images", class_mapping)
print("\n")
print("Validation set class distribution:")
count_labels_per_class("dataset/split/val/labels", "dataset/split/val/images", class_mapping)
print("\n")
print("Test set class distribution:")
count_labels_per_class("dataset/split/test/labels", "dataset/split/test/images", class_mapping)

Training set class distribution before sampling:
Total object counts by class:
Class 0 ("apple"): 240
Class 1 ("avocado"): 230
Class 2 ("banana"): 8
Class 5 ("orange"): 111
Class 9 ("watermelon"): 450


Validation set class distribution:
Total object counts by class:
Class 0 ("apple"): 54
Class 1 ("avocado"): 48
Class 2 ("banana"): 3
Class 5 ("orange"): 28
Class 9 ("watermelon"): 84


Test set class distribution:
Total object counts by class:
Class 0 ("apple"): 36
Class 1 ("avocado"): 29
Class 5 ("orange"): 16
Class 9 ("watermelon"): 58


In [34]:
# Undersample/oversample to balance classes in training set
img_class_map_train = count_image_classes("dataset/split/train/labels")
undersampled = balance_classes(img_class_map_train, target_num_per_class=300, seed=SEED)
# Save undersampled training set
save_balanced_split(
    undersampled,
    "dataset/split/train/labels",
    "dataset/split/train/images",
    "dataset/split/train/balanced_labels",
    "dataset/split/train/balanced_images"
)

KeyboardInterrupt: 

In [11]:
# Oversample (duplicate) in the balanced train folder
oversample_classes(
    selected_files=os.listdir("dataset/split/train/balanced_labels"),
    image_class_map=count_image_classes("dataset/split/train/balanced_labels"),
    class_target=300,
    output_images="dataset/split/train/balanced_images",
    output_labels="dataset/split/train/balanced_labels",
    labels_folder="dataset/split/train/balanced_labels",
    images_folder="dataset/split/train/balanced_images"
)

Oversampling complete. New class distribution: Counter({1: 301, 8: 301, 9: 300, 6: 300, 0: 300, 2: 300, 5: 300, 3: 300, 4: 300, 7: 300})


#### Dataset summary

In [33]:
print("Training set class distribution after sampling:")
count_labels_per_class("dataset/split/train/balanced_labels", "dataset/split/train/balanced_images", class_mapping)
print("")
print("Validation set class distribution:")
count_labels_per_class("dataset/split/val/labels", "dataset/split/val/images", class_mapping)
print("\n")
print("Test set class distribution:")
count_labels_per_class("dataset/split/test/labels", "dataset/split/test/images", class_mapping)

Training set class distribution after sampling:


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/split/train/balanced_labels'